# Agent playground
![image.png](https://mintcdn.com/langchain-5e9cc07a/-_xGPoyjhyiDWTPJ/oss/images/agent.png?w=840&fit=max&auto=format&n=-_xGPoyjhyiDWTPJ&q=85&s=bd932835b919f5e58be77221b6d0f194)

In [30]:
from os import environ
environ['CUDA_VISIBLE_DEVICES'] = '1'

In [31]:
!pip install langchain langchain_openai langchain_tavily

# Initialize the base language model

In [32]:
local_inference = False

### A) Cloud inference 
1. via [*Hugging Face’s Inference Providers*](https://huggingface.co/docs/inference-providers/en/index)
    - Create an account for the Hugging Face platform: [huggingface.co/join](https://huggingface.co/join)
    - Get the API key from dashboard: [huggingface.co/docs/hub/en/security-tokens](https://huggingface.co/docs/hub/en/security-tokens)
2. via [*OpenAI API*](https://auth.openai.com)
    - Create a new OpenAI account (free credits): [https://auth.openai.com/log-in](https://auth.openai.com/log-in)
    - Generate the API key from the dashboard: [platform.openai.com/api-keys](https://platform.openai.com/api-keys)

In [ ]:
from os import environ
environ["HF_TOKEN"] = "" 
environ["OPENAI_API_KEY"] = ""

In [34]:
from langchain_openai import ChatOpenAI

# Cloud inference via OpenAI
if not local_inference and environ.get('OPENAI_API_KEY'):
    llm_model = ChatOpenAI(model="gpt-5-nano", api_key=environ["OPENAI_API_KEY"])
    
    print(f"Cloud inference ({llm_model.openai_api_base}): model: \"{llm_model.model_name}\"")
    
# Cloud inference via HuggingFace
elif not local_inference and environ.get('HF_TOKEN'):
    llm_model = ChatOpenAI(
        base_url="https://router.huggingface.co/v1",
        model="Qwen/Qwen3-Next-80B-A3B-Instruct", # (1 )Qwen/Qwen3-Next-80B-A3B-Instruct || (2) openai/gpt-oss-120b
        api_key=environ["HF_TOKEN"])
    
    print(f"Cloud inference ({llm_model.openai_api_base}): model: \"{llm_model.model_name}\"")

Cloud inference (https://router.huggingface.co/v1): model: "Qwen/Qwen3-Next-80B-A3B-Instruct"


### B) Local inference

In [35]:
from langchain_huggingface import HuggingFacePipeline, ChatHuggingFace

if local_inference:
    llm_model = ChatHuggingFace(
        llm = HuggingFacePipeline.from_model_id(
            model_id="allenai/OLMo-2-0425-1B-Instruct", #  Qwen/Qwen3-4B-Instruct-2507
            task ="text-generation",
            pipeline_kwargs={'dtype':"bfloat16"}
        ))
    print(f"Local Inference: \"{llm_model.llm.model_id}\"")

# Initialize the tool
This code demonstrates how to use the `TavilySearch` tool from the `langchain_tavily` package to perform a web search within a LangChain workflow. 
1. It imports the TavilySearch class, which is a tool designed to query the Tavily Search API and return structured search results, such as URLs, snippets, and optionally images or answers.
2. The invoke method is then called with the query `"What is Italy’s current public debt?"`. 
    - This method sends the query to the Tavily API and returns the search results as a dictionary containing information such as the original query, a list of result items (with titles, URLs, and content snippets), and possibly other metadata.
3. The results are printed to the output pane, allowing you to inspect the returned data. 

The code also shows how to organize tools for later use by placing the search tool into a list called tools. This is useful when building more complex agent workflows that may use multiple tools for different tasks.

In [36]:
from langchain_tavily import TavilySearch
from json import dumps

# Initialize the Tavily Search tool
search_tool = TavilySearch(max_results=2, tavily_api_key = "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")

# Try out the search tool
#search_results = search_tool.invoke(input = "What is Italy’s current public debt?")
#print(dumps(search_results, indent=4))

# Invoke the agent with a user query


In [37]:
query = "What's the weather like today in Trento, Italy?"

### A) without the search tool

In [38]:
from langchain.agents import create_agent
agent_executor = create_agent(
    model = llm_model)
    #system_prompt = "You are a helpful assistant that exploits all available tools to find up-to-date information.")

In [39]:
# Define the input message
input_message = {"messages": {"role": "user", "content": query}}

# Invoke the agent
response = agent_executor.invoke(input_message)

# Print the response
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

What's the weather like today in Trento, Italy?
================================== Ai Message ==================================

I can't provide real-time weather updates. To find the current weather in Trento, Italy, I recommend checking a reliable weather service such as:

- [Weather.com](https://www.weather.com)
- [AccuWeather](https://www.accuweather.com)
- [Meteo.it](https://www.meteo.it) (Italian meteorological service)
- Your smartphone’s weather app

Simply search “Trento Italy weather today” for the most accurate and up-to-date information, including temperature, precipitation, wind, and forecast details. Enjoy your day in Trento — it’s a beautiful city in the Trentino-Alto Adige region! 🌞🏔️


### B) with the search tool

In [40]:
agent_executor = create_agent(
    model = llm_model, 
    tools = [search_tool],
    system_prompt = "You are a helpful assistant that exploits all available tools to find up-to-date information.")

In [41]:
# Define the input message
input_message = {"messages": {"role": "user", "content": query}}

# Invoke the agent
response = agent_executor.invoke(input_message)

# Print the response
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

What's the weather like today in Trento, Italy?
================================== Ai Message ==================================

I'll check the current weather in Trento, Italy for you.
Tool Calls:
  tavily_search (call_43367fd17f9f4a79a6fcc6)
 Call ID: call_43367fd17f9f4a79a6fcc6
  Args:
    query: current weather in Trento Italy
    search_depth: basic
================================= Tool Message =================================
Name: tavily_search

{"query": "current weather in Trento Italy", "follow_up_questions": null, "answer": null, "images": [], "results": [{"title": "Weather in Trento, Italy", "url": "https://www.weatherapi.com/", "content": "{'location': {'name': 'Trento', 'region': 'Trentino-Alto Adige', 'country': 'Italy', 'lat': 46.0667, 'lon': 11.1333, 'tz_id': 'Europe/Rome', 'localtime_epoch': 1762439477, 'localtime': '2025-11-06 15:31'}, 'current': {'last_updated_epoch': 1762439400, 'l

# Create our custom tools

In [42]:
def get_exam_score(exam_name: str) -> dict:
    """Get the expected score for a given exam."""
    
    # For demonstration purposes, we assume a perfect score (we have high expectations!)
    student_score = 30
    
    return {
        'exam_name': exam_name, 
        'range': (0, 30), 
        'score': student_score}

def parse_result(score: int) -> dict:
    """Get the expected result (pass or fail) for a given score."""
    
    # For demonstration purposes, we assume a traditional passing threshold
    pass_threshold = 18
    
    # Context: exam is graded out of 30, with 18 as the passing threshold
    results = {
        'score': score, 
        'pass_threshold': pass_threshold,
        'passed': score >= pass_threshold,
        'cum_laude': False # sorry :/
    }
    
    return results

In [43]:
agent_executor = create_agent(
    model = llm_model, 
    tools = [get_exam_score, parse_result],
    system_prompt = "You are a helpful assistant that exploits all available tools to find up-to-date information.")

In [44]:
# Wait for 5 seconds to avoid rate limiting issues
import time 
time.sleep(5)

In [45]:
query = 'Will I ever pass the FM 2025 exam?'

In [46]:
response = agent_executor.invoke({"messages": {"role": "user", "content": query}})

# Print the response
for message in response["messages"]:
    message.pretty_print()

APIStatusError: Error code: 402 - {'error': 'You have exceeded your monthly included credits for Inference Providers. Subscribe to PRO to get 20x more monthly included credits.'}

# Human in the loop

In [ ]:
def parse_result(score: int) -> dict:
    """Get the expected result (pass or fail) for a given score."""
    
    # For demonstration purposes, we assume a traditional passing threshold
    pass_threshold = 18
    
    # Ask for human approval if the score is passing
    accepted = None
    if score >= pass_threshold:
        user_input = input(f"Do you accept a score equal to {score} (yes/no): ").strip().lower()
        accepted = True if user_input == 'yes' else False
    
    # Context: exam is graded out of 30, with 18 as the passing threshold
    results = {
        'score': score, 
        'pass_threshold': pass_threshold,
        'passed': score >= pass_threshold,
        'cum_laude': False, # sorry :/
        'acceptedByStudent': accepted
    }
    
    return results

In [69]:
agent_executor = create_agent(
    model = llm_model, 
    tools = [get_exam_score, parse_result],
    system_prompt = "You are a helpful assistant that exploits all available tools to find up-to-date information.")

In [70]:
response = agent_executor.invoke({"messages": {"role": "user", "content": query}}, config = {"configurable": {"thread_id": "101"}})

# Print the response
for message in response["messages"]:
    message.pretty_print()

================================ Human Message =================================

Will I ever pass the FM 2025 exam?
================================== Ai Message ==================================
Tool Calls:
  get_exam_score (call_bb95988e5e154f4facd289)
 Call ID: call_bb95988e5e154f4facd289
  Args:
    exam_name: FM 2025
================================= Tool Message =================================
Name: get_exam_score

{"exam_name": "FM 2025", "range": [0, 30], "score": 30}
================================== Ai Message ==================================
Tool Calls:
  parse_result (call_6d218a97add64b44aace3a)
 Call ID: call_6d218a97add64b44aace3a
  Args:
    score: 30
================================= Tool Message =================================
Name: parse_result

{"score": 30, "pass_threshold": 18, "passed": true, "cum_laude": false, "accepted": false}
================================== Ai Message ==================================

Based on the information available, you are